In [ ]:
%pip install --extra-index-url https://pypi.org/simple --upgrade --quiet np_workflows
%load_ext autoreload
%autoreload 2
from __future__ import annotations

import datetime
import logging
import os
import pathlib
import time

import IPython

import np_workflows.workflows.shared.npxc as npxc
import np_session
import np_config
import np_logging

from np_workflows.services import utils
from np_workflows.services import open_ephys as OpenEphys
from np_workflows.services.proxies import (
    Sync, ImageMVR, NoCamstim, NoCamstim, NewScaleCoordinateRecorder,
)
from np_workflows.services.protocols import (
    Pretestable, Initializable, Startable, Testable, Stoppable, Finalizable, Pretestable, Verifiable, Validatable,
    TestError,
    )

logging.getLogger('np_config').setLevel(logging.INFO)
logger = np_logging.getLogger()
logger.setLevel(logging.INFO)

npxc.start_rsc_apps()

In [ ]:
mouse = 654442
user ='andrew.shelton'

In [ ]:
data_root = pathlib.Path('//allen/programs/mindscope/workgroups/np-ultra')
session_folder = data_root / f'{datetime.datetime.now():%Y-%m-%d}_{mouse}'

session_folder.mkdir(parents=True, exist_ok=True)

services = (Sync, ImageMVR, OpenEphys, NewScaleCoordinateRecorder, NoCamstim)

# *Pretest*

In [ ]:
OpenEphys.pretest()

In [ ]:
ImageMVR.pretest()

In [ ]:
NewScaleCoordinateRecorder.pretest()

In [ ]:
Sync.pretest()

In [ ]:
NoCamstim.remote_file = 'c:/users/svc_neuropix/Desktop/run_opto_pretest.bat'

NewScaleCoordinateRecorder.log_root = session_folder
NoCamstim.password = 'ti8*DYne'

NoCamstim.start() #! needs biscript_pretest_v2 on np3

# *Experiment*

#  **Before putting mouse on stage**

**Open Ephys**
* Record Nodes set to A:/B: (or E:/G: on NP.3)
* Disk space on each Record Node (single long vertical bar)
* 50/50 channels on each Record Node (shorter vertical bars) #!
* Tip-reference on all probes
* Barcodes displaying

**Stage**
* Probes fully retracted, cartridge raised
* Water lines flushed

In [ ]:
NoCamstim.remote_file = 'c:/users/svc_neuropix/Desktop/run_blue_opto.bat'

NewScaleCoordinateRecorder.log_root = session_folder
NoCamstim.password = 'ti8*DYne'

trial_idx = None

## **Initialize & test services**

In [ ]:
for service in services:
    if isinstance(service, Initializable):
        service.initialize()
    if isinstance(service, Testable):
        service.test()

## **Photodoc of probes in dye**

# **Load mouse onto stage**

Before lowering cartridge, check:
* Stabilization screw
* quickcast, agarose/silicon oil
* Tail cone down
* Continuity check


# **Lower cartridge**

## **Photodoc of brain surface**

Turn on light and focus on brain surface

In [ ]:
def photodoc(label: str | None) -> pathlib.Path:
    ImageMVR.label = label
    ImageMVR.start()
    ImageMVR.finalize()
    NewScaleCoordinateRecorder.label = label
    NewScaleCoordinateRecorder.start()
    return ImageMVR.data_files[-1]

IPython.display.Image(photodoc('pre_experiment_brain_surface_image'))

# **Insert probes**

In [ ]:
from np_probe_targets.implant_drawing import ProbeInsertionsTS5, ProbeTargetInsertionRecordWidget
ProbeInsertionsTS5.save_dir = session_folder
targets = ProbeInsertionsTS5([None] * 6)
IPython.display.display(ProbeTargetInsertionRecordWidget(targets))

# *Start of trial loop*

In [ ]:
trial_idx = 0 if trial_idx is None else trial_idx + 1

OpenEphys.set_folder(f'{session_folder.name}_{trial_idx}')

## **Advance probes to target depth**

## **Start settle timer**

In [ ]:
wait_sec = 10 * 60

time_0 = time.time()
time_remaining = lambda: datetime.timedelta(seconds = wait_sec - (time.time() - time_0))
while time_remaining().total_seconds() > 0:
    print(f'{time_remaining()}', end='\r', flush=True)
    time.sleep(.1)

## **Photodoc after probes reach target depth**

Turn on light and focus on brain surface

In [ ]:
IPython.display.Image(photodoc(f'pre_recording_image_{trial_idx}'))

# **Start recording**

### Update recording directory name in OpenEphys!

In [ ]:
for service in (Sync, OpenEphys):
    service.start()
    if isinstance(service, Verifiable):
        time.sleep(1)
        service.verify()

# **Start opto trial**

In [ ]:
logger.info('Starting stimulus')

NoCamstim.start()

while not NoCamstim.is_ready_to_start():
    time.sleep(10)

if isinstance(NoCamstim, Finalizable):
    NoCamstim.finalize()

## **Stop recording**

In [ ]:
for service in (_ for _ in (Sync, OpenEphys) if isinstance(_, Stoppable)):
    service.stop()

for service in (_ for _ in (Sync, OpenEphys) if isinstance(_, Finalizable)):
    service.finalize()

for service in (_ for _ in (Sync, OpenEphys) if isinstance(_, Validatable)):
    service.validate()

# *End of trial loop*

## **Fully retract probes**

## **Final photodoc of brain surface without probes**

Turn on light and focus on brain surface

In [ ]:
IPython.display.Image(photodoc('post_experiment_brain_surface_image'))

## **Finish up**

* raise cartridge
* add quickcast
* remove and water mouse

## **Copy files**

In [ ]:
import shutil
import contextlib
for service in services:
    with contextlib.suppress(AttributeError):
        for file in service.data_files:
            print(service.data_files)
            shutil.copy2(file, session_folder)